In [ ]:
import ray
import pandas as pd


### Import Methods

#### Way 1

without directly using the
`from .predictability.utils import get_column_combinations`
in `src/asd/__init__.py` file.

In [ ]:
'''
import asd.predictability.utils as asdpu
asdpu.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=4,
                                  outputs=1,
                                  targets=[5, 6]
                                  )
'''

#### Way 2

with using the import
`from .predictability.utils import get_column_combinations`
in `src/asd/__init__.py` file.

In [ ]:
import asd

In [ ]:
asd.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=4,
                                  outputs=1,
                                  targets=[5, 6]
                                  )

In [ ]:
ray.init()

# Basic Example

## Load Data

Basic data that can be understood as two different input columns (linear and random values) and three different output columns (sin of linear x, 1 + sin of linear x and random values).

In [ ]:
data = pd.read_csv("example_data.csv", dtype=float)

In [ ]:
data.head()

## Get Combinations function

This function can be used to determine the overall number of combinations the predictability routine analyses given the number of data columns, fitting type etc. This allows to estimate the overall runtime of the predictability routine.

The function returns a list of combination tuples, where the first inputs-many elements correspond to the inputs and the remaining ouputs-many to the targets. The argument targets can be used to define columns that should exclusively be regarded as targets.

In [ ]:
# applied to a numerical example:
asd.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=3,
                                  outputs=1,
                                  targets=[5, 6]
                                  )

In [ ]:
# the argument "amount_only" can be used to output the amout of combinations only
asd.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=3,
                                  outputs=1,
                                  targets=[5, 6],
                                  amount_only=True
                                  )

In [ ]:
# applied to the data above
print(asd.get_column_combinations(all_cols=data.columns, inputs=1, outputs=1, targets=["sinx", "randomy", "sinx_plus1"]))

In [ ]:
print(asd.get_column_combinations(all_cols=data.columns, inputs=1, outputs=1, targets=["sinx", "randomy", "sinx_plus1"],
                                  amount_only=True))

## Predictability function

Running the predictability function over all possible 1+1 combinations where the respective target is either sin, sin+1 or random and the input is linear x or random values.

The purpose of having sin + 1 is to have a tuple that exclusively contains positive values so fitting a power law can be applied.

### "normal" routine

In [ ]:
metrics_dict, datas_dict = asd.run_predictability(data=data,
                                          input_cols=1,
                                          output_cols=1,
                                          col_set=None,
                                          targets=["sinx", "randomy", "sinx_plus1"],
                                          method="kNN",
                                          random_state_split=None,
                                          #refined=True
                                          )

_Note that starting the Ray instance accounts for most of the runtime. Start the cell above again and runtime will be ~1s._

### greedy routine

In [ ]:
greedy_metrics_dict, greedy_datas_dict = asd.run_predictability(data=data,
                                          input_cols=2,
                                          output_cols=1,
                                          col_set=None,
                                          targets=["sinx", "randomy", "sinx_plus1"],
                                          method="kNN",
                                          random_state_split=None,
                                          #refined=True,
                                          greedy=True
                                          )

### Results

In [ ]:
pd.DataFrame.from_dict(metrics_dict).transpose()

In [ ]:
pd.DataFrame.from_dict(greedy_metrics_dict).transpose()

### Structure of returned dictionaries

In [ ]:
struc_dict = datas_dict[list(datas_dict.keys())[0]]
for key in list(struc_dict.keys()):
    if type(struc_dict[key]) is dict:
        print(key, "\t dict with key(s):\t", list(struc_dict[key].keys()))
    else:
        print(key, "\t type:\t", type(struc_dict[key]), "\t shape:\t", struc_dict[key].shape)

### Plotting

In [ ]:
asd.predictability.utils.plot_result(datas_dict, list(datas_dict.keys())[0], plot_along=["linear", "mean"])

## Tuple Selection function

This function can be used to limit the number of tuples that is further analysed in more detail.

In [ ]:
selected_tuples = asd.predictability.core.tuple_selection(metrics_dict, n_best=2)
selected_tuples

## Refine Predictability function

This function is used to further refine the predictability of the previously selected best tuples.

In [ ]:
refined_metrics_dict, refined_datas_dict = asd.refine_predictability(best_tuples=selected_tuples,
                                                                 data_dict=datas_dict,
                                                                 time_left_for_this_task=60,
                                                                 per_run_time_limit=30,
                                                                 n_jobs=-1,
                                                                 use_ray=True,
                                                                 )

In [ ]:
pd.DataFrame.from_dict(refined_metrics_dict).transpose()

#### Structure of returned dictionaries

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
struc_dict = refined_datas_dict[list(refined_datas_dict.keys())[0]]
for key in list(struc_dict.keys()):
    if type(struc_dict[key]) is dict:
        print(key, "\t dict with key(s):\t"#, list(struc_dict[key].keys())
              )
    elif type(struc_dict[key]) is Pipeline:
        print(key, "\t type:\t", type(struc_dict[key]))
    else:
        print(key, "\t type:\t", type(struc_dict[key]), "\t shape:\t", struc_dict[key].shape)

#### Plotting

In [ ]:

for key in list(refined_datas_dict.keys()):
    refined_datas_dict[key]["y_test_pred_init"] = datas_dict[key]["y_test_pred"]
    refined_datas_dict[key]["y_test_init"] = datas_dict[key]["y_test"]
    refined_datas_dict[key]["y_test_pred_linear"] = datas_dict[key]["y_test_pred_linear"]
    refined_datas_dict[key]["y_test_pred_mean"] = datas_dict[key]["y_test_pred_mean"]
    #refined_datas_dict[key]["y_test_init"] = datas_dict[key]["y_test"]


In [ ]:
asd.predictability.utils.plot_result(refined_datas_dict, list(refined_datas_dict.keys())[0], plot_along=[#"linear", "mean",
                                                                                                         "init"])

#### Structure of returned dictionaries

In [23]:
from sklearn.pipeline import Pipeline

In [24]:
struc_dict = refined_datas_dict[list(refined_datas_dict.keys())[0]]
for key in list(struc_dict.keys()):
    if type(struc_dict[key]) is dict:
        print(key, "\t dict with key(s):\t"#, list(struc_dict[key].keys())
              )
    elif type(struc_dict[key]) is Pipeline:
        print(key, "\t type:\t", type(struc_dict[key]))
    else:
        print(key, "\t type:\t", type(struc_dict[key]), "\t shape:\t", struc_dict[key].shape)

X_train 	 type:	 <class 'numpy.ndarray'> 	 shape:	 (69, 1)
X_test 	 type:	 <class 'numpy.ndarray'> 	 shape:	 (30, 1)
y_train 	 type:	 <class 'numpy.ndarray'> 	 shape:	 (69, 1)
y_test 	 type:	 <class 'numpy.ndarray'> 	 shape:	 (30,)
y_train_pred 	 type:	 <class 'numpy.ndarray'> 	 shape:	 (69,)
y_test_pred 	 type:	 <class 'numpy.ndarray'> 	 shape:	 (30,)
ensemble 	 type:	 <class 'sklearn.pipeline.Pipeline'>
pareto_pipelines 	 dict with key(s):	
all_individuals 	 dict with key(s):	


#### Plotting

In [25]:

for key in list(refined_datas_dict.keys()):
    refined_datas_dict[key]["y_test_pred_init"] = datas_dict[key]["y_test_pred"]
    refined_datas_dict[key]["y_test_init"] = datas_dict[key]["y_test"]
    refined_datas_dict[key]["y_test_pred_linear"] = datas_dict[key]["y_test_pred_linear"]
    refined_datas_dict[key]["y_test_pred_mean"] = datas_dict[key]["y_test_pred_mean"]
    #refined_datas_dict[key]["y_test_init"] = datas_dict[key]["y_test"]


In [27]:
asd.predictability.utils.plot_result(refined_datas_dict, list(refined_datas_dict.keys())[0], plot_along=[#"linear", "mean",
                                                                                                         "init"])